In [1]:
import pandas as pd
import numpy as np
import os
from ete3 import Tree

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 20 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
ploidb_path = "/groups/itay_mayrose/halabikeren/PloiDB/chromevol/results/with_model_weighting/ploidb_by_genus_without_missing.csv"
species_path = ploidb_path # "/groups/itay_mayrose/halabikeren/plant_pollinator_networks/data/name_resolution/resolved_plant_names.csv"
ploidb_pipeline_dir = "/groups/itay_mayrose/halabikeren/PloiDB/chromevol/with_model_weighting/by_genus_on_unresolved_ALLMB_and_unresolved_ccdb/"

In [3]:
ploidb = pd.read_csv(ploidb_path)
ploidb = ploidb[[c for c in ploidb.columns if "Unnamed" not in c]]

In [4]:
if species_path == ploidb_path:
    species_df = ploidb
else:
    species_df = pd.read_csv(species_path)
    species_df = species_df.dropna(subset=["resolved_name"]).merge(ploidb, left_on="resolved_name", right_on="Taxon", how="left")

In [5]:
species_df_by_genus = species_df.groupby("Genus")
genera = list(species_df_by_genus.groups.keys())

In [12]:
def get_polyloidy_age(genus_df: pd.DataFrame) -> float:
    genus_df["polyploid_age"] = np.nan
    genus = genus_df.Genus.str.lower().values[0]
    genus_tree_path = f"{ploidb_pipeline_dir}{genus}/chromevol/model_selection/gain_loss/MLAncestralReconstruction.tree"
    genus_tree = Tree(genus_tree_path, format=1)
    polyploidy_ages_path = f"{ploidb_pipeline_dir}{genus}/chromevol/observed_polyploidization_events.csv"
    if not os.path.exists(polyploidy_ages_path):
        return genus_df
    try:
        polyploidy_ages = pd.read_csv(polyploidy_ages_path)
    except Exception as e:
        print(f"failed to read ploidy ages data from {polyploidy_ages_path} due to error {e}")
        return genus_df
    polyploidy_ages = polyploidy_ages.loc[polyploidy_ages.event_type.isin(["DUPLICATION", "DEMI-DUPLICATION", "BASE-NUMBER"])]
    polyploids_to_age = polyploidy_ages.groupby("branch_child_name")["age"].max().to_dict() # age is measured by distance from leaves (time=0) so higher value means order. we want to measure the polyploid age according to the age of the first polyploidization event leading to it
    for node in genus_tree.traverse("preorder"):
        node_name = "-".join(node.name.split("-")[:-1])
        if node_name in polyploids_to_age:
            descendents = ["-".join(l.split("-")[:-1]) for l in node.get_leaf_names()]
            genus_df.loc[(genus_df.Taxon.isin(descendents)) & (genus_df.polyploid_age.isna()), "polyploid_age"] = polyploids_to_age[node_name]
     
    return genus_df

In [13]:
species_df_with_ploidy_ages = pd.concat([species_df.groupby("Genus").apply(get_polyloidy_age)])

/tmp/ipykernel_10858/1169179207.py:10: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  polyploidy_ages = pd.read_csv(polyploidy_ages_path)
/tmp/ipykernel_10858/1169179207.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  polyploidy_ages = pd.read_csv(polyploidy_ages_path)


failed to read ploidy ages data from /groups/itay_mayrose/halabikeren/PloiDB/chromevol/with_model_weighting/by_genus_on_unresolved_ALLMB_and_unresolved_ccdb/suaeda/chromevol/observed_polyploidization_events.csv due to error Error tokenizing data. C error: Expected 12 fields in line 73, saw 24



In [ ]:
species_df_with_ploidy_ages.to_csv("